In [2]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp, get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [4]:
file_path = "../../../data/processed_data/force_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_outer, category_force_outer, category_names, df_force_outer = get_final_df(data, is_acc_data=0, is_multilabel=1)

labels = category_force_outer.copy()
features=features_force_outer.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (1330, 1)
Labels Shape:  (1330, 4)


In [5]:
max_length = 150
for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
feat= np.zeros(len(features) *max_length).reshape(-1, max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (1330, 150)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [7]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1197, 1, 150)
(1197, 4)
(133, 1, 150)
(133, 4)


In [8]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [10]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#2: Force Outer",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-09 03:40:19,711] A new study created in RDB with name: Case#2: Force Outer


Epoch 10/50, Training Loss: 0.6538609027862549, Validation Loss: 0.6508051156997681
Epoch 20/50, Training Loss: 0.6294435103734334, Validation Loss: 0.6275395601987839
Epoch 30/50, Training Loss: 0.6285265326499939, Validation Loss: 0.626141294836998
Epoch 40/50, Training Loss: 0.6285502235094707, Validation Loss: 0.6259065270423889


[I 2023-10-09 03:40:46,803] Trial 0 finished with value: 0.6259816288948059 and parameters: {'lr': 0.00035041330149065004, 'dropout_prob': 0.18706466857294485, 'hidden_channels': 131, 'n_layers': 7, 'kernel_size': 4, 'stride': 7, 'padding': 10, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 0 with value: 0.6259816288948059.


Epoch 50/50, Training Loss: 0.6285241842269897, Validation Loss: 0.6259816288948059
Epoch 10/50, Training Loss: 0.6955217242240905, Validation Loss: 0.6865777373313904
Epoch 20/50, Training Loss: 0.6865099390347799, Validation Loss: 0.6773219108581543
Epoch 30/50, Training Loss: 0.6780013759930928, Validation Loss: 0.6687121540307999
Epoch 40/50, Training Loss: 0.6700621008872986, Validation Loss: 0.6606368720531464


[I 2023-10-09 03:40:49,732] Trial 1 finished with value: 0.6530462950468063 and parameters: {'lr': 1.2862187543451002e-05, 'dropout_prob': 0.011449780355891559, 'hidden_channels': 63, 'n_layers': 1, 'kernel_size': 4, 'stride': 2, 'padding': 7, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 0 with value: 0.6259816288948059.


...

Epoch 50/50, Training Loss: 0.0006102788157780499, Validation Loss: 0.0006085487048039795
Epoch 10/50, Training Loss: 0.010940518125426024, Validation Loss: 0.0023757539493090007
Epoch 20/50, Training Loss: 0.006184109684545547, Validation Loss: 0.006125386571511626
Epoch 30/50, Training Loss: 0.00017350168879299115, Validation Loss: 0.00028579708828146977
Epoch 40/50, Training Loss: 2.5736299357959068e-05, Validation Loss: 0.0007772434840944697


[I 2023-10-09 04:21:46,009] Trial 98 finished with value: 0.0008121513620835685 and parameters: {'lr': 0.004464368212321376, 'dropout_prob': 0.23122089154240288, 'hidden_channels': 185, 'n_layers': 4, 'kernel_size': 8, 'stride': 6, 'padding': 7, 'pooling_type': 'max', 'use_batch_norm': True}. Best is trial 75 with value: 1.943814652349829e-05.


Epoch 50/50, Training Loss: 1.6242548720886892e-05, Validation Loss: 0.0008121513620835685
Epoch 10/50, Training Loss: 0.016988670577605565, Validation Loss: 0.024114715633913875
Epoch 20/50, Training Loss: 0.004595679882913828, Validation Loss: 0.005639554132358171
Epoch 30/50, Training Loss: 0.008294080562579135, Validation Loss: 0.04440965736284852
Epoch 40/50, Training Loss: 0.0016722764521546196, Validation Loss: 0.026794276433065534


[I 2023-10-09 04:22:07,131] Trial 99 finished with value: 0.00766369748453144 and parameters: {'lr': 0.001554464081698639, 'dropout_prob': 0.2616769743949508, 'hidden_channels': 211, 'n_layers': 5, 'kernel_size': 9, 'stride': 5, 'padding': 8, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 75 with value: 1.943814652349829e-05.


Epoch 50/50, Training Loss: 0.012263689128061136, Validation Loss: 0.00766369748453144


In [11]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.00435612106373003, 'dropout_prob': 0.21769773768647277, 'hidden_channels': 223, 'n_layers': 4, 'kernel_size': 7, 'stride': 5, 'padding': 9, 'pooling_type': 'max', 'use_batch_norm': True}
Best final validation loss:
1.943814652349829e-05


In [12]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/force_outer_best_configs.json"

In [14]:

# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [13]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You